In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
r = pd.read_csv('goodbooks-10k-dataset/ratings.csv')
tr = pd.read_csv('goodbooks-10k-dataset/to_read.csv')
b = pd.read_csv('goodbooks-10k-dataset/books.csv')

t = pd.read_csv('goodbooks-10k-dataset/tags.csv')
bt = pd.read_csv('goodbooks-10k-dataset/book_tags.csv')
bg = pd.read_csv('goodbooks-10k-dataset/book_genre.csv')

In [3]:
from sklearn.model_selection import train_test_split

# Prepare ratings data
ratings = r[['user_id', 'book_id', 'rating']].copy()

# Split into train and test (80/20)
train_ratings, test_ratings = train_test_split(ratings, test_size=0.2, random_state=42)

# Further split test into validation and test if needed
# val_ratings, test_ratings = train_test_split(test_ratings, test_size=0.5, random_state=42)

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

# Create user-item matrix
user_item_matrix = train_ratings.pivot_table(index='user_id', columns='book_id', values='rating')

# Normalize ratings: subtract user mean
user_means = user_item_matrix.mean(axis=1)
normalized_matrix = user_item_matrix.sub(user_means, axis=0).fillna(0)

similarity_matrix = cosine_similarity(normalized_matrix)
similarity_matrix = pd.DataFrame(similarity_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)

In [5]:
from sklearn.metrics import ndcg_score

def evaluate_recommendations(test_ratings, recommendations, k=5):
    """
    Evaluate recommendations against test set
    
    Args:
        test_ratings: DataFrame with true user-item ratings
        recommendations: Dict {user_id: list of recommended book_ids}
        k: Top-k recommendations to evaluate
    
    Returns:
        Dictionary of evaluation metrics
    """
    # Create a test set of user-item pairs with ratings >= threshold (e.g., 4)
    test_positives = test_ratings[test_ratings['rating'] >= 4]
    test_dict = test_positives.groupby('user_id')['book_id'].apply(list).to_dict()
    
    precision_scores = []
    recall_scores = []
    ndcg_scores = []
    
    for user_id, recs in recommendations.items():
        if user_id not in test_dict:
            continue
            
        true_positives = test_dict[user_id]
        if not true_positives:
            continue
            
        # Get top-k recommendations
        top_k_recs = recs[:k]
        
        # Calculate hits
        hits = len(set(top_k_recs) & set(true_positives))
        
        # Precision@k
        precision = hits / k
        precision_scores.append(precision)
        
        # Recall@k
        recall = hits / len(true_positives)
        recall_scores.append(recall)
        
        # NDCG@k (we'll use binary relevance here)
        relevance = [1 if book in true_positives else 0 for book in top_k_recs]
        ideal_relevance = sorted(relevance, reverse=True)
        if sum(ideal_relevance) > 0:
            ndcg = ndcg_score([ideal_relevance], [relevance], k=k)
            ndcg_scores.append(ndcg)
    
    return {
        'precision@k': np.mean(precision_scores) if precision_scores else 0,
        'recall@k': np.mean(recall_scores) if recall_scores else 0,
        'ndcg@k': np.mean(ndcg_scores) if ndcg_scores else 0,
        'coverage': len(recommendations) / test_ratings['user_id'].nunique(),
        'num_users_evaluated': len(precision_scores)
    }

In [6]:
predicted_ratings = similarity_matrix.dot(user_item_matrix.fillna(0))

In [7]:
denominator = np.abs(similarity_matrix).sum(axis=1).values.reshape(-1, 1)
predicted_ratings = predicted_ratings / (denominator + 1e-8)  # add epsilon to avoid divide-by-zero

In [44]:
import random
from tqdm import tqdm

# Convert prediction matrix back to DataFrame
predicted_df = pd.DataFrame(predicted_ratings, index=user_item_matrix.index, columns=user_item_matrix.columns)

# Filter test users to those who appear in training
test_users = test_ratings['user_id'].unique()
train_users = set(train_ratings['user_id'].unique())
valid_users = list(set(test_users) & train_users)

# Generate recommendations
recommendations = {}
for user_id in tqdm(valid_users, desc="Generating recommendations"):
    if user_id not in predicted_df.index:
        continue

    user_preds = predicted_df.loc[user_id]
    already_read = user_item_matrix.loc[user_id].notnull()
    already_read = already_read[already_read].index.values
    # Exclude already-read books
    recs = user_preds[~user_preds.index.isin(already_read)]
    top_recs = recs.sort_values(ascending=False).head(20).index.tolist()
    recommendations[user_id] = top_recs

# Evaluate
metrics = evaluate_recommendations(test_ratings, recommendations, k=10)
print("Evaluation Metrics:")
print(f"Precision: {metrics['precision@k']:.4f}")
print(f"Recall: {metrics['recall@k']:.4f}")
print(f"NDCG: {metrics['ndcg@k']:.4f}")
print(f"Coverage: {metrics['coverage']:.2%}")
print(f"Users evaluated: {metrics['num_users_evaluated']}")

Generating recommendations: 100%|████████| 53423/53423 [02:15<00:00, 393.25it/s]


Evaluation Metrics:
Precision: 0.0899
Recall: 0.1204
NDCG: 0.5647
Coverage: 100.00%
Users evaluated: 53349


In [45]:
# Evaluate
metrics = evaluate_recommendations(test_ratings, recommendations, k=10)
print("Evaluation Metrics:")
print(f"Precision: {metrics['precision@k']:.4f}")
print(f"Recall: {metrics['recall@k']:.4f}")
print(f"NDCG: {metrics['ndcg@k']:.4f}")
print(f"Coverage: {metrics['coverage']:.2%}")
print(f"Users evaluated: {metrics['num_users_evaluated']}")

Evaluation Metrics:
Precision: 0.1213
Recall: 0.0824
NDCG: 0.6645
Coverage: 100.00%
Users evaluated: 53349
